# Homework Week 1

In [5]:
import pandas as pd
import pickle
from pathlib import Path

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [14]:
def read_dataframe(filename):
    if filename.endswith(".csv"):
        df = pd.read_csv(filename)

        # df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        # df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith(".parquet"):
        df = pd.read_parquet(filename)

    # df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    # df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    # df = df[(df.duration >= 1) & (df.duration <= 60)]

    # categorical = ["PULocationID", "DOLocationID"]
    # df[categorical] = df[categorical].astype(str)

    return df

In [15]:
data_dir = Path.cwd().parent.parent.parent.joinpath("data")
data_dir

WindowsPath('h:/mlops-zoomcamp/data')

In [39]:
df_train = read_dataframe(str(data_dir.joinpath("yellow_tripdata_2023-01.parquet")))
df_val = read_dataframe(str(data_dir.joinpath("yellow_tripdata_2023-02.parquet")))

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [40]:
len(df_train.columns)

19

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

In [41]:
df_train.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [42]:
df_train["duration"] = df_train.tpep_dropoff_datetime - df_train.tpep_pickup_datetime
df_train["duration"] = df_train.duration.apply(lambda td: td.total_seconds() / 60)

In [48]:
df_val["duration"] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val["duration"] = df_val.duration.apply(lambda td: td.total_seconds() / 60)

In [43]:
df_train.duration.std()

42.59435124195458

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [44]:
total_len = len(df_train)

In [49]:
df_train = df_train[(df_train.duration >= 1) & (df_train.duration <= 60)]
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

In [47]:
len(df_train)/ total_len

0.9812202822125979

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [50]:
categorical = ['PULocationID', 'DOLocationID']
df_train[categorical] = df_train[categorical].astype(str)
df_val[categorical] = df_val[categorical].astype(str)

In [28]:
df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]


In [51]:
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

In [64]:
df_train.shape

(3009173, 20)

In [34]:
categorical = ["PU_DO"]  #'PULocationID', 'DOLocationID']
numerical = ["trip_distance"]

dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)


In [62]:
len(train_dicts)

3066766

In [52]:

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [63]:
X_train.shape

(3066766, 22755)

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [53]:
target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

In [61]:
X_train.shape

(3066766, 22755)

In [59]:
len(y_train)

3009173

In [54]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

ValueError: Found input variables with inconsistent numbers of samples: [3066766, 3009173]